# Kasra Samadi 993623030

### Import Libraries

In [1]:
#!pip3 install torch

In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import struct

### Hyperparameters

In [3]:
num_epochs = 10
batch_size = 64
learning_rate = 0.001

### Function to read IDX file format

In [4]:
def read_idx(file_path):
    with open(file_path, 'rb') as f:
        zero, data_type, dims = struct.unpack('>HBB', f.read(4))
        shape = tuple(struct.unpack('>I', f.read(4))[0] for d in range(dims))
        return np.frombuffer(f.read(), dtype=np.uint8).reshape(shape)

### Load MNIST dataset from IDX files

In [5]:
train_images = read_idx('train-images.idx3-ubyte').copy()
train_labels = read_idx('train-labels.idx1-ubyte').copy()
test_images = read_idx('t10k-images.idx3-ubyte').copy()
test_labels = read_idx('t10k-labels.idx1-ubyte').copy()

### Convert data to tensors and normalize

In [6]:
train_images_tensor = torch.from_numpy(train_images).float() / 255.0
train_labels_tensor = torch.from_numpy(train_labels).long()
test_images_tensor = torch.from_numpy(test_images).float() / 255.0
test_labels_tensor = torch.from_numpy(test_labels).long()

### Create TensorDatasets

In [7]:
train_dataset = TensorDataset(train_images_tensor, train_labels_tensor)
test_dataset = TensorDataset(test_images_tensor, test_labels_tensor)

### Data loaders

In [8]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Neural network model

In [9]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(784, 256)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

### Set device and create model

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)

### Loss function and optimizer

In [11]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

### Train the model

In [12]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 100 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}], Step [{i + 1}/{total_steps}], Loss: {loss.item():.6f}")


Epoch [1/10], Step [100/938], Loss: 0.428581
Epoch [1/10], Step [200/938], Loss: 0.296483
Epoch [1/10], Step [300/938], Loss: 0.223102
Epoch [1/10], Step [400/938], Loss: 0.448439
Epoch [1/10], Step [500/938], Loss: 0.114073
Epoch [1/10], Step [600/938], Loss: 0.188511
Epoch [1/10], Step [700/938], Loss: 0.104128
Epoch [1/10], Step [800/938], Loss: 0.102333
Epoch [1/10], Step [900/938], Loss: 0.289269
Epoch [2/10], Step [100/938], Loss: 0.190454
Epoch [2/10], Step [200/938], Loss: 0.061856
Epoch [2/10], Step [300/938], Loss: 0.097737
Epoch [2/10], Step [400/938], Loss: 0.138280
Epoch [2/10], Step [500/938], Loss: 0.151021
Epoch [2/10], Step [600/938], Loss: 0.227541
Epoch [2/10], Step [700/938], Loss: 0.203237
Epoch [2/10], Step [800/938], Loss: 0.125374
Epoch [2/10], Step [900/938], Loss: 0.082955
Epoch [3/10], Step [100/938], Loss: 0.089556
Epoch [3/10], Step [200/938], Loss: 0.043912
Epoch [3/10], Step [300/938], Loss: 0.087853
Epoch [3/10], Step [400/938], Loss: 0.141124
Epoch [3/1

### Test the model

In [13]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    predictions = []

    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        predictions.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.5f} %")

Test Accuracy: 97.78000 %


### Convert predictions to a DataFram

In [14]:
df = pd.DataFrame({'Class': predictions})

### Save DataFrame to CSV file

In [15]:
df.to_csv('Q4.csv', index=False)